In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from nflows.transforms.dequantization import UniformDequantization
from nflows.transforms.dequantization import VariationalDequantization

In [4]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [5]:
# Generate some dist with 2 real and 2 discrete dims

# These are the probabilities to draw a particular label
x_prob_1 = np.random.rand(10)
x_prob_2 = np.random.rand(5)
p_labels_1 = x_prob_1/(np.sum(x_prob_1))
p_labels_2 = x_prob_2/(np.sum(x_prob_2))
p_cum_labels_1 = np.cumsum(p_labels_1)
p_cum_labels_2 = np.cumsum(p_labels_2)

def p(x,y):
    if np.all(x > 0) and np.all(x < 1) and x.shape == y.shape:
        return m.exp(-(x[0] + x[1])/2) * np.cos((x[0]*y[0] + x[1]*y[1])*m.pi)
    return 0

def generate(n):
    x = np.zeros((n,4))

    counter = 0
    while(counter < n):
        y = np.zeros(2)
        y[0] = np.argmax(r.random() < p_cum_labels_1)
        y[1] = np.argmax(r.random() < p_cum_labels_2)

        while(True):
            # Get a point
            x_trial = np.random.rand(2)
            if r.random() < p(x_trial, y):
                x[counter][0] = x_trial[0]
                x[counter][1] = x_trial[1]
                x[counter][2] = y[0]
                x[counter][3] = y[1]
                counter += 1
                break

    return x

In [6]:
num_layers = 6
base_dist_uniform = BoxUniform(torch.zeros(4), torch.ones(4))
base_dist_variational = BoxUniform(torch.zeros(4), torch.ones(4))

transforms_uniform = []
transforms_variational = []

transforms_uniform.append(UniformDequantization(max_labels=torch.tensor([-1,-1,9,4])))
transforms_variational.append(VariationalDequantization(max_labels=torch.tensor([-1,-1,9,4]), rqs_hidden_features=15, rqs_flow_layers=2))

for _ in range(num_layers):
    transforms_uniform.append(RandomPermutation(features=4))
    transforms_uniform.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=4, 
        hidden_features=25,
        num_bins=10,
        num_blocks=4,
    ))

    transforms_variational.append(RandomPermutation(features=4))
    transforms_variational.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=4, 
        hidden_features=25,
        num_bins=10,
        num_blocks=4,
    ))

transform_uniform = CompositeTransform(transforms_uniform)
transform_variational = CompositeTransform(transforms_variational)

flow_uniform = Flow(transform_uniform, base_dist_uniform).to(device)
flow_variational = Flow(transform_variational, base_dist_variational).to(device)

optimizer_uniform = optim.Adam(flow_uniform.parameters())
optimizer_variational = optim.Adam(flow_variational.parameters())

In [7]:
x_data = torch.tensor(generate(1000000), dtype=torch.float32, device=device)

In [ ]:
n_epochs = 200
batch_size = 10000
n_batches = m.ceil(x_data.shape[0]/batch_size)

for epoch in range(n_epochs):
    permutation = torch.randperm(x_data.shape[0], device=device)    

    # Loop over batches
    cum_loss_uniform = 0
    cum_loss_variational = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, x_data.shape[0]-1 )
        indices = permutation[batch_begin:batch_end]
        batch_x = x_data[indices]
        
        # Take a step
        optimizer_uniform.zero_grad()
        optimizer_variational.zero_grad()

        loss_uniform = -flow_uniform.log_prob(inputs=batch_x).mean()
        loss_variational = -flow_variational.log_prob(inputs=batch_x).mean()

        loss_uniform.backward()
        loss_variational.backward()

        optimizer_uniform.step()
        optimizer_variational.step()

        # Compute cumulative loss
        cum_loss_uniform = (cum_loss_uniform*batch + loss_uniform.item())/(batch+1)
        cum_loss_variational = (cum_loss_variational*batch + loss_variational.item())/(batch+1)

        print("epoch = ", epoch, "batch = ",batch+1, "/", n_batches, "loss_uniform = ", cum_loss_uniform, " loss_variational = ", cum_loss_variational)

epoch =  0 batch =  1 / 100 loss_uniform =  1.582231044769287  loss_variational =  2.492642402648926
epoch =  0 batch =  2 / 100 loss_uniform =  1.38106769323349  loss_variational =  2.1916006803512573
epoch =  0 batch =  3 / 100 loss_uniform =  1.2138011852900188  loss_variational =  1.9817252953847249
epoch =  0 batch =  4 / 100 loss_uniform =  1.07146917283535  loss_variational =  1.8473657965660095
epoch =  0 batch =  5 / 100 loss_uniform =  0.9553629636764527  loss_variational =  1.7413829803466796
epoch =  0 batch =  6 / 100 loss_uniform =  0.8554205248753229  loss_variational =  1.6503669023513794
epoch =  0 batch =  7 / 100 loss_uniform =  0.767932972737721  loss_variational =  1.5556705934660775
epoch =  0 batch =  8 / 100 loss_uniform =  0.69587542116642  loss_variational =  1.4679329842329025
epoch =  0 batch =  9 / 100 loss_uniform =  0.6343493925200568  loss_variational =  1.390655431482527
epoch =  0 batch =  10 / 100 loss_uniform =  0.5773375533521176  loss_variational =

epoch =  0 batch =  80 / 100 loss_uniform =  -0.3550507990788901  loss_variational =  -0.11824675541720345
epoch =  0 batch =  81 / 100 loss_uniform =  -0.35887741510480364  loss_variational =  -0.12453099279836916
epoch =  0 batch =  82 / 100 loss_uniform =  -0.3627600610034722  loss_variational =  -0.13074891709313696
epoch =  0 batch =  83 / 100 loss_uniform =  -0.36642512842081487  loss_variational =  -0.1366555543941815
epoch =  0 batch =  84 / 100 loss_uniform =  -0.37006910838508267  loss_variational =  -0.1427361472410016
epoch =  0 batch =  85 / 100 loss_uniform =  -0.37370881246786347  loss_variational =  -0.14862304193460757
epoch =  0 batch =  86 / 100 loss_uniform =  -0.37743177578167253  loss_variational =  -0.15450040168172216
epoch =  0 batch =  87 / 100 loss_uniform =  -0.3809682348470107  loss_variational =  -0.16021489381426188
epoch =  0 batch =  88 / 100 loss_uniform =  -0.38421099143240345  loss_variational =  -0.16562335298460137
epoch =  0 batch =  89 / 100 loss

epoch =  1 batch =  58 / 100 loss_uniform =  -0.7221398446066626  loss_variational =  -0.7077041535541929
epoch =  1 batch =  59 / 100 loss_uniform =  -0.7222381082631774  loss_variational =  -0.7078205367266122
epoch =  1 batch =  60 / 100 loss_uniform =  -0.7227619876464207  loss_variational =  -0.7084029803673426
epoch =  1 batch =  61 / 100 loss_uniform =  -0.7230418285385507  loss_variational =  -0.7089217881687352
epoch =  1 batch =  62 / 100 loss_uniform =  -0.7234429476722595  loss_variational =  -0.7094385354749618
epoch =  1 batch =  63 / 100 loss_uniform =  -0.7239238704953875  loss_variational =  -0.7099449435869852
epoch =  1 batch =  64 / 100 loss_uniform =  -0.724268832243979  loss_variational =  -0.7104651406407356
epoch =  1 batch =  65 / 100 loss_uniform =  -0.7244652848977309  loss_variational =  -0.7107786306968102
epoch =  1 batch =  66 / 100 loss_uniform =  -0.7245587959434047  loss_variational =  -0.7110332214471066
epoch =  1 batch =  67 / 100 loss_uniform =  -0

epoch =  2 batch =  37 / 100 loss_uniform =  -0.7574932897413099  loss_variational =  -0.7596290417619653
epoch =  2 batch =  38 / 100 loss_uniform =  -0.7573242611006686  loss_variational =  -0.7598619163036346
epoch =  2 batch =  39 / 100 loss_uniform =  -0.7576739314274911  loss_variational =  -0.760525141006861
epoch =  2 batch =  40 / 100 loss_uniform =  -0.7577849537134171  loss_variational =  -0.7608972877264023
epoch =  2 batch =  41 / 100 loss_uniform =  -0.758006627966718  loss_variational =  -0.7616320281493955
epoch =  2 batch =  42 / 100 loss_uniform =  -0.7579977739424933  loss_variational =  -0.7619760433832804
epoch =  2 batch =  43 / 100 loss_uniform =  -0.7577606176221093  loss_variational =  -0.7619523475336474
epoch =  2 batch =  44 / 100 loss_uniform =  -0.7579785896973177  loss_variational =  -0.7624523070725527
epoch =  2 batch =  45 / 100 loss_uniform =  -0.758374281724294  loss_variational =  -0.7630819585588243
epoch =  2 batch =  46 / 100 loss_uniform =  -0.7

epoch =  3 batch =  15 / 100 loss_uniform =  -0.7844016273816427  loss_variational =  -0.8478205760320028
epoch =  3 batch =  16 / 100 loss_uniform =  -0.7848221063613892  loss_variational =  -0.8484725952148438
epoch =  3 batch =  17 / 100 loss_uniform =  -0.7844063253963695  loss_variational =  -0.8487028234145221
epoch =  3 batch =  18 / 100 loss_uniform =  -0.7852840390470293  loss_variational =  -0.8492476642131805
epoch =  3 batch =  19 / 100 loss_uniform =  -0.7855808734893799  loss_variational =  -0.8504702982149626
epoch =  3 batch =  20 / 100 loss_uniform =  -0.7860862761735916  loss_variational =  -0.8510920464992523
epoch =  3 batch =  21 / 100 loss_uniform =  -0.786317663533347  loss_variational =  -0.8515110072635469
epoch =  3 batch =  22 / 100 loss_uniform =  -0.7868207693099976  loss_variational =  -0.8522330766374414
epoch =  3 batch =  23 / 100 loss_uniform =  -0.7876813204392142  loss_variational =  -0.8536406729532324
epoch =  3 batch =  24 / 100 loss_uniform =  -0

...

epoch =  138 batch =  61 / 100 loss_uniform =  -1.472913022901191  loss_variational =  -1.560118016649465
epoch =  138 batch =  62 / 100 loss_uniform =  -1.4731424970011557  loss_variational =  -1.5601770916292745
epoch =  138 batch =  63 / 100 loss_uniform =  -1.473219706898644  loss_variational =  -1.560313031786964
epoch =  138 batch =  64 / 100 loss_uniform =  -1.4732057508081198  loss_variational =  -1.5600864831358194
epoch =  138 batch =  65 / 100 loss_uniform =  -1.4731577102954572  loss_variational =  -1.5602610918191764
epoch =  138 batch =  66 / 100 loss_uniform =  -1.4729927669871936  loss_variational =  -1.5600534894249656
epoch =  138 batch =  67 / 100 loss_uniform =  -1.4733142443557283  loss_variational =  -1.5602480617921743
epoch =  138 batch =  68 / 100 loss_uniform =  -1.473302357337054  loss_variational =  -1.5603468575898338
epoch =  138 batch =  69 / 100 loss_uniform =  -1.473461351532867  loss_variational =  -1.5601562866266223
epoch =  138 batch =  70 / 100 los

epoch =  139 batch =  38 / 100 loss_uniform =  -1.4728474271924872  loss_variational =  -1.5586937634568463
epoch =  139 batch =  39 / 100 loss_uniform =  -1.4724879081432636  loss_variational =  -1.5582690789149356
epoch =  139 batch =  40 / 100 loss_uniform =  -1.4724762916564942  loss_variational =  -1.5584255576133725
epoch =  139 batch =  41 / 100 loss_uniform =  -1.4728232069713314  loss_variational =  -1.5588489567361223
epoch =  139 batch =  42 / 100 loss_uniform =  -1.473407112416767  loss_variational =  -1.5592039369401474
epoch =  139 batch =  43 / 100 loss_uniform =  -1.4733699698780858  loss_variational =  -1.5592206855152926
epoch =  139 batch =  44 / 100 loss_uniform =  -1.4737761345776645  loss_variational =  -1.5595434091307896
epoch =  139 batch =  45 / 100 loss_uniform =  -1.4735096163219876  loss_variational =  -1.559032943513658
epoch =  139 batch =  46 / 100 loss_uniform =  -1.4737321164297021  loss_variational =  -1.5590635227120437
epoch =  139 batch =  47 / 100

epoch =  140 batch =  15 / 100 loss_uniform =  -1.480141774813334  loss_variational =  -1.5687808672587076
epoch =  140 batch =  16 / 100 loss_uniform =  -1.4794403463602066  loss_variational =  -1.5679546296596527
epoch =  140 batch =  17 / 100 loss_uniform =  -1.4783370635088753  loss_variational =  -1.5670707576415117
epoch =  140 batch =  18 / 100 loss_uniform =  -1.4778252906269498  loss_variational =  -1.5660345355669658
epoch =  140 batch =  19 / 100 loss_uniform =  -1.477002363455923  loss_variational =  -1.56527200498079
epoch =  140 batch =  20 / 100 loss_uniform =  -1.4761622905731202  loss_variational =  -1.5647350311279298
epoch =  140 batch =  21 / 100 loss_uniform =  -1.476308754512242  loss_variational =  -1.5649574938274564
epoch =  140 batch =  22 / 100 loss_uniform =  -1.4759860905733975  loss_variational =  -1.5648185177282854
epoch =  140 batch =  23 / 100 loss_uniform =  -1.4761948792830757  loss_variational =  -1.5649456666863484
epoch =  140 batch =  24 / 100 lo

epoch =  140 batch =  93 / 100 loss_uniform =  -1.4718380423002346  loss_variational =  -1.5646425267701507
epoch =  140 batch =  94 / 100 loss_uniform =  -1.471422755971868  loss_variational =  -1.5646705018713118
epoch =  140 batch =  95 / 100 loss_uniform =  -1.4714968731528835  loss_variational =  -1.5647598090924715
epoch =  140 batch =  96 / 100 loss_uniform =  -1.4715888140102227  loss_variational =  -1.5650833348433177
epoch =  140 batch =  97 / 100 loss_uniform =  -1.4713667176433445  loss_variational =  -1.5650220777570587
epoch =  140 batch =  98 / 100 loss_uniform =  -1.4712952618696251  loss_variational =  -1.5651943951236957
epoch =  140 batch =  99 / 100 loss_uniform =  -1.4711470736397638  loss_variational =  -1.5651358968079692
epoch =  140 batch =  100 / 100 loss_uniform =  -1.470974761247635  loss_variational =  -1.5650070059299468
epoch =  141 batch =  1 / 100 loss_uniform =  -1.448041558265686  loss_variational =  -1.5501153469085693
epoch =  141 batch =  2 / 100 l

epoch =  141 batch =  71 / 100 loss_uniform =  -1.4646021060540642  loss_variational =  -1.5613249137368002
epoch =  141 batch =  72 / 100 loss_uniform =  -1.4646188004149332  loss_variational =  -1.5612699306673474
epoch =  141 batch =  73 / 100 loss_uniform =  -1.4651462411227292  loss_variational =  -1.5614731426108373
epoch =  141 batch =  74 / 100 loss_uniform =  -1.4655306000967283  loss_variational =  -1.5616329602293066
epoch =  141 batch =  75 / 100 loss_uniform =  -1.4659659322102865  loss_variational =  -1.5618614339828492
epoch =  141 batch =  76 / 100 loss_uniform =  -1.46608672800817  loss_variational =  -1.5618718768420972


In [ ]:
n_sample = 100000
with torch.no_grad():
    x_uniform = flow_uniform.sample(n_sample).cpu()
    x_variational = flow_variational.sample(n_sample).cpu()

In [ ]:
plt.hist(x_data.cpu()[:n_sample,2].numpy(), histtype='stepfilled', edgecolor="black", facecolor="lightgray", bins=np.linspace(-0.5, 9.5, 11))
plt.hist(x_uniform[:,2].numpy(), edgecolor="red", histtype="step", bins = np.linspace(-0.5, 9.5, 11))
plt.hist(x_variational[:,2].numpy(), edgecolor="green", histtype="step", bins = np.linspace(-0.5, 9.5, 11))
plt.show()

In [ ]:
with torch.no_grad():
    x_uniform_dequantized, _ = flow_uniform._transform._transforms[0].forward(x_data)
    x_variational_dequantized, _ = flow_variational._transform._transforms[0].forward(x_data)

In [ ]:
plt.hist(x_uniform_dequantized[:,2].cpu().numpy(), histtype='stepfilled', edgecolor="black", facecolor="lightgray", bins = 100)
plt.hist(x_variational_dequantized[:,2].cpu().numpy(), edgecolor="red", histtype="step", bins = 100)
plt.show()